In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from typing import List
import pandas as pd
import os

In [2]:
urlmi = "https://www.flipkart.com/mobiles/mi~brand/pr?sid=tyy,4io&otracker=nmenu_sub_Electronics_0_Mi&otracker=nmenu_sub_Electronics_0_Mi"
urloppo = "https://www.flipkart.com/search?q=oppo+mobile&sid=tyy%2C4io&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_1_3_na_na_ps&otracker1=AS_QueryStore_OrganicAutoSuggest_1_3_na_na_ps&as-pos=1&as-type=RECENT&suggestionId=oppo+mobile+5g%7CMobiles&requestId=5b6557ac-a8da-4f08-a79f-247224ab9704&as-backfill=on"
urlsamsung = "https://www.flipkart.com/search?p%5B%5D=facets.brand%255B%255D%3DSamsung&sid=tyy%2F4io&sort=recency_desc&ctx=eyJjYXJkQ29udGV4dCI6eyJhdHRyaWJ1dGVzIjp7InRpdGxlIjp7Im11bHRpVmFsdWVkQXR0cmlidXRlIjp7ImtleSI6InRpdGxlIiwiaW5mZXJlbmNlVHlwZSI6IlRJVExFIiwidmFsdWVzIjpbIkxhdGVzdCBTYW1zdW5nIG1vYmlsZXMgIl0sInZhbHVlVHlwZSI6Ik1VTFRJX1ZBTFVFRCJ9fX19fQ%3D%3D&wid=1.productCard.PMU_V2_1"
urlvivo = "https://www.flipkart.com/mobiles/pr?sid=tyy%2C4io&p%5B%5D=facets.serviceability%5B%5D%3Dfalse&p%5B%5D=facets.brand%255B%255D%3DVivo&otracker=nmenu_sub_Electronics_0_Vivo"
urlrealme = "https://www.flipkart.com/mobiles/pr?sid=tyy%2C4io&p%5B%5D=facets.brand%255B%255D%3DRealme&otracker=nmenu_sub_Electronics_0_Realme"

url_list = [urlmi,urloppo, urlsamsung, urlvivo, urlrealme]

company_list = ["mi", "oppo", "samsung", "vivo", "realme"]

In [3]:
class Mobile:
    def __init__(self, company:str=None,title:str=None,price:int=None,ratings:str=None,status:str=None,imageUrl:str=None,redirectUrl:str=None):
        self.company = company
        self.title = title
        self.price = price
        self.ratings = ratings
        self.status = status
        self.imageUrl = imageUrl
        self.redirectUrl = redirectUrl

In [4]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
page_count = []
def getPageCount(url):
    driver.get(url)
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")
    element = soup.find('div', attrs={'class': '_1G0WLw'})
    page_count.append(int(element.span.text[10:len(element.span.text)]))

for i in range(0, len(url_list)):
    try:
        getPageCount(url_list[i])
    except Exception as e:
        print("Error occured while fetch page coun for url :"+ url_list[i] + " Exception : " + e)
driver.close()

In [5]:
print(page_count)

[16, 5, 35, 20, 30]


In [7]:
cd = Mobile()
for i in range(0,len(url_list)):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    cellData : List[Mobile]=[]
    cd.company = company_list[i]    
    try:
        for j in range(0,page_count[i]):
            driver.get(url_list[i]+"&page="+str(j))
            html_content = driver.page_source
            soup = BeautifulSoup(html_content, "html.parser")
            mainelement = soup.find_all('a', attrs={'class': 'CGtC98'})
            for ele in mainelement:
                if ele is not None:
                    # Title
                    t = ele.find(
                        'div', attrs={'class': 'KzDlHZ'})
                    cd.title = '' if t is None or t.text is None else t.text
                    print(cd.title)
                    # Price
                    t = ele.find(
                        'div', attrs={'class': 'Nx9bqj _4b5DiR'})
                    cd.price = '' if t is None or t.text is None else t.text
                    # Ratings
                    t = ele.find('span', attrs={'class': 'Wphh3N'})
                    cd.ratings = '' if t is None or t.span is None or t.span.text is None else t.span.text
                    # Status
                    t = ele.find('span', attrs={'class': 'vfSpSs'})
                    if t is None or t.span is None or t.span.text is None:
                        cd.status = "Available"
                    else:
                        cd.status = t.span.text
                    # ImageUrl
                    t = ele.find('img', attrs={'class': 'DByuf4'})
                    cd.imageUrl = '' if t is None else t['src']
                    # RedirectUrl
                    cd.redirectUrl = '' if ele['href'] is None else "https://www.flipkart.com"+ele['href']
                    cellData.append(Mobile(cd.company,cd.title,cd.price,cd.ratings,cd.status,cd.imageUrl,cd.redirectUrl))
                else:
                   print("Element is none "+ str(ele))
        # Making seperate csv company wise
        try:
            df = pd.DataFrame([vars(mb) for mb in cellData])
            dict ="E:/P-projects/scrap-celldata/scrap_selenium/Files"
            df.to_csv(os.path.join(dict,cd.company+".csv"),index=False)
        except Exception as e:
            print("Error occured while making csv for company :"+cd.company+ " Exception : "+str(e))
    except Exception as e:
        print("Something went wrong during scrap data with company : "+cd.company+" Exception : "+str(e))          
    driver.close()
    del cellData
del cd

Xiaomi 14 CIVI (Matcha Green, 256 GB)
Xiaomi 14 CIVI (Hot Pink, 512 GB)
Xiaomi 14 CIVI (Shadow Black, 512 GB)
Xiaomi 14 CIVI (Cruise Blue, 256 GB)
Xiaomi 14 CIVI (Shadow Black, 256 GB)
Xiaomi 11T Pro 5G Hyperphone (Moonlight White, 256 GB)
Xiaomi 14 CIVI (Panda White, 512 GB)
Xiaomi 14 CIVI (Matcha Green, 512 GB)
Mi 11 Lite (Vinyl Black, 128 GB)
Xiaomi 11T Pro 5G Hyperphone (Moonlight White, 256 GB)
Xiaomi 14 CIVI (Aqua Blue, 512 GB)
Xiaomi 14 CIVI (Cruise Blue, 512 GB)
Xiaomi 11T Pro 5G Hyperphone (Celestial Magic, 256 GB)
Xiaomi 11T Pro 5G Hyperphone (Moonlight White, 128 GB)
Mi 11 Lite (Jazz Blue, 128 GB)
Mi 11 Lite (Tuscany Coral, 128 GB)
Redmi 9 (Carbon Black, 64 GB)
Mi 11 Lite (Vinyl Black, 128 GB)
Redmi 9A Sport (Coral Green, 32 GB)
Xiaomi 11T Pro 5G Hyperphone (Celestial Magic, 128 GB)
Redmi 9A Sport (Carbon Black, 32 GB)
Redmi 9A Sport (Metallic Blue, 32 GB)
Xiaomi 11T Pro 5G Hyperphone (Meteorite Black, 256 GB)
Xiaomi 11T Pro 5G Hyperphone (Meteorite Black, 128 GB)
Xiaomi 14 